In [2]:
pip install mlrose

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn.metrics import precision_score,accuracy_score,f1_score,recall_score,roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
from sklearn.ensemble import VotingClassifier
from sklearn import preprocessing
from mlxtend.classifier import StackingClassifier
import matplotlib.gridspec as gridspec
import itertools
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics

In [4]:
# loading out dataset from previous notebook
urldata = pd.read_csv("/content/Url_Processed.csv")

# droping "Unnamed: 0" as its unncessary feature
urldata.drop("Unnamed: 0",axis=1,inplace=True)


# remove uneccessary columns
urldata.drop(["url","label"],axis=1,inplace=True)

urldata.head()
     

,result,url_length,hostname_length,path_length,fd_length,count-,count@,count?,count%,count.,count=,count-http,count-https,count-www,count-digits,count-letters,count_dir,use_of_ip,short_url
0,0,22,14,0,0,0,0,0,0,2,0,1,1,1,0,17,0,1,1
1,0,23,15,0,0,0,0,0,0,2,0,1,1,1,0,18,0,1,1
2,0,24,16,0,0,0,0,0,0,2,0,1,1,1,0,19,0,1,1
3,0,21,13,0,0,0,0,0,0,2,0,1,1,1,0,16,0,1,1
4,0,25,17,0,0,0,0,0,0,2,0,1,1,1,0,20,0,1,1


In [5]:
#Oversampling using SMOTE

# NOTE : We'll use only lexical features for this notebook i,e not taking 'url_length' and 'short_url'

#Independent Variables
x = urldata[['hostname_length',
       'path_length', 'fd_length', 'count-', 'count@', 'count?',
       'count%', 'count.', 'count=', 'count-http','count-https', 'count-www', 'count-digits',
       'count-letters', 'count_dir', 'use_of_ip']]

#Dependent Variable
y = urldata['result']


In [6]:
from imblearn.over_sampling import SMOTE

x_sample, y_sample = SMOTE().fit_resample(x, y.values.ravel())

x_sample = pd.DataFrame(x_sample)
y_sample = pd.DataFrame(y_sample)
# DATA SPLITTING 

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_sample, y_sample, test_size = 0.2)

In [7]:
DT = DecisionTreeClassifier(max_depth=7)
RF = RandomForestClassifier(max_depth=7)
NB=BernoulliNB()

In [8]:
classifier_labels = ["RF","DT","NB"]
classifiers = {
               "DT":DT,
               "RF":RF,
               "NB":NB,
               }
combo_classifiers = []
for ii in range(2, len(classifier_labels)+1):
    for subset in itertools.combinations(classifier_labels, ii):
        combo_classifiers.append(subset)
print(combo_classifiers)

[('RF', 'DT'), ('RF', 'NB'), ('DT', 'NB'), ('RF', 'DT', 'NB')]


In [9]:
for combo in combo_classifiers:
    # Get labels of classifier to create a stack
    labels = list(combo)
    classifier_combo = []
    for ii in range(len(labels)):
        
        label = combo[ii]
        classifier = classifiers[label]
        classifier_combo.append(classifier)
    
    
    #print(classifier_combo)
    sclf = StackingClassifier(classifiers = classifier_combo, 
                         meta_classifier=RF)
    scores = cross_val_score(sclf, x, y, cv=2, scoring='accuracy').mean()
    print(combo ,scores )

('RF', 'DT') 0.9910168467443844
('RF', 'NB') 0.9911367998294001
('DT', 'NB') 0.9910146253909582
('RF', 'DT', 'NB') 0.9910679378731874
